<a href="https://colab.research.google.com/github/Tharindupriyaharshana/H-rverstehenPro/blob/blender_bot/Eleuther410m_blender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fine-tune the EleutherAI Pythia-410M model to develop a chatbot specialized in answering questions related to Blender software**

In [ ]:
! pip install accelerate -U
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 25.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
pip install lamini==2.0.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00


In [ ]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.1 MB/s eta 0:00:00


In [ ]:
import lamini
from llama import BasicModelRunner

import datasets
from datasets import load_dataset
from datasets import Dataset

import transformers
from transformers import pipeline


import itertools
import jsonlines

from pprint import pprint

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import pandas as pd

In [ ]:
lamini.api_key = "398b57c2bb67727dd3e5c8f52120da25d0127c5de5085dca66ff31daa71fd683"

In [ ]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd
import jsonlines
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from llama import BasicModelRunner
logger = logging.getLogger(__name__)
global_config = None

In [ ]:
model_name = "EleutherAI/pythia-410m"     #"meta-llama/Llama-2-7b-chat-hf"

In [ ]:
def tokenize_function(examples):
    # Concatenate questions and answers if they exist, otherwise use 'text' or 'input'
    if "question" in examples and "answer" in examples:
        text = [q + a for q, a in zip(examples["question"], examples["answer"])]
    elif "input" in examples and "output" in examples:
        text = [i + o for i, o in zip(examples["input"], examples["output"])]
    else:
        text = examples["text"]

    tokenizer.pad_token = tokenizer.eos_token
    tokenized_inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=2048,
        return_tensors="pt"
    )

    return tokenized_inputs


In [ ]:
finetuning_dataset_loaded = datasets.load_dataset('Fikrat/blender', split='train')
tokenizer = AutoTokenizer.from_pretrained(model_name)  #tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
tokenized_dataset = finetuning_dataset_loaded.map(
    tokenize_function,
    batched=True,
)

print(tokenized_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/106 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/106 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask'],
    num_rows: 106
})


In [ ]:
tokenized_dataset = tokenized_dataset.add_column("labels", tokenized_dataset["input_ids"])

In [ ]:
df = pd.DataFrame(tokenized_dataset)
df.head()

,input,output,input_ids,attention_mask,labels
0,How do you add a 3D element to your scene afte...,Go to the compositing tab to change additional...,"[2347, 513, 368, 823, 247, 495, 37, 3284, 281,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2347, 513, 368, 823, 247, 495, 37, 3284, 281,..."
1,What tab do you need to go to for changing set...,To change settings related to the final render...,"[1276, 10334, 513, 368, 878, 281, 564, 281, 32...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1276, 10334, 513, 368, 878, 281, 564, 281, 32..."
2,How can you render a transparent sequence of P...,Delete some nodes in the compositing tab and a...,"[2347, 476, 368, 8600, 247, 13955, 3425, 273, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2347, 476, 368, 8600, 247, 13955, 3425, 273, ..."
3,What does adding a plane to your scene and set...,Adding a plane and setting it as a shadow catc...,"[1276, 1057, 6240, 247, 6415, 281, 634, 6200, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1276, 1057, 6240, 247, 6415, 281, 634, 6200, ..."
4,"In Blender, what key combination allows precis...","While moving in the viewport, holding Shift an...","[688, 2071, 3109, 13, 752, 2234, 5019, 4483, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[688, 2071, 3109, 13, 752, 2234, 5019, 4483, 1..."


In [ ]:
#this is the splititng part after the preperation
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, shuffle=True, seed=123)
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 95
    })
    test: Dataset({
        features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11
    })
})


In [ ]:
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(model_name)   #base_model = GPT2LMHeadModel.from_pretrained('gpt2-xl')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/911M [00:00<?, ?B/s]

In [ ]:
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

In [ ]:
base_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
  

In [ ]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100): # max input token and minimum input limit may have there. For different models limit can change
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [ ]:
test_text = test_dataset[0]['input']
print("Question input (test):", test_text)
print(f"Correct answer from blender docs: {test_dataset[0]['output']}")
print("Model's answer: ")
print(inference(test_text, base_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): How can I change the interpolation handles for multiple keyframes simultaneously in Blender?
Correct answer from blender docs: To change interpolation handles for multiple keyframes at once, select all the keyframes, press 'T' to bring up the interpolation menu, and choose from presets like 'consistent,' 'linear,' or 'bza' (Bezier). This allows for efficient and comprehensive adjustments to the animation flow.
Model's answer: 


I have a Blender file with multiple keyframes. I want to change the interpolation handles for multiple keyframes simultaneously.

A:

You can use the Blender Interpolation API to do this.

Open the Blender file.
Select the keyframe you want to change.
Select the "Interpolation" tab.
Select the "Interpolation" option.


In [ ]:
max_steps = 95
"""A step is a batch of training data.
And so if your batch size is one, it's just one data point. If
your batch size is 2,000, it's 2,000 data points."""

"A step is a batch of training data.\nAnd so if your batch size is one, it's just one data point. If\nyour batch size is 2,000, it's 2,000 data points."

In [ ]:
trained_model_name = f"blender{max_steps}_steps"
output_dir = trained_model_name

In [ ]:
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=1,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=20, # Number of update steps between two evaluations
  save_steps=20, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

In [ ]:
"""load this up in the trainer class.
This is a class we wrapped around HuggingFaces
main trainer class basically doing the same thing
just printing out things for you as you train and as you
can see you put a few things in"""

from transformers import Trainer

trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
training_output = trainer.train()

Step,Training Loss,Validation Loss
20,0.012300,0.117572
40,0.019500,0.117539
60,0.009000,0.118550
80,0.004400,0.122065


In [ ]:
save_dir = f'{output_dir}/final'

trainer.save_model(save_dir)
print("Saved model to:", save_dir)

Saved model to: blender3_steps/final


In [ ]:
finetuned_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)
finetuned_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
  

In [ ]:
test_question = test_dataset[0]['input']
print("Question input (test):", test_question)
print("Finetuned model's answer: ")
print(inference(test_question, finetuned_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): How can I change the interpolation handles for multiple keyframes simultaneously in Blender?
Finetuned model's answer: 
To change interpolation handles for multiple keyframes at once, select all the keyframes, press 'T' to bring up the interpolation menu, and choose from presets like 'consistent,' 'linear,' or 'bza' (Bezier). This allows for flexible and consistent interpolation between keyframes.


In [ ]:
test_answer = test_dataset[0]['output']
print("Target answer output (test):", test_answer)

Target answer output (test): To change interpolation handles for multiple keyframes at once, select all the keyframes, press 'T' to bring up the interpolation menu, and choose from presets like 'consistent,' 'linear,' or 'bza' (Bezier). This allows for efficient and comprehensive adjustments to the animation flow.
